# 1. 自动识别

In [1]:
import os

In [23]:
list_file = list(os.walk('./input'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
for file in list_file:
    if '整理好的替代料总表' in file:
        alternate = './input/' + file
    elif '改价表' in file:
        stock = './input/' + file
    elif '替代料总表' in file and '整理好的' not in file:
        combination = './input/' + file
    elif 'MRP' in file:
        mrp = './input/' + file
    elif '划分' in file:
        group = './input/' + file
    elif '断货改价' in file:
        last = './input/' + file

print('alternate:     ' + alternate)
print('stock:         ' + stock)
print('combination:   ' + combination)
print('mrp:           ' + mrp)
print('group:         ' + group)
print('last:          ' + last)

alternate:     ./input/整理好的替代料总表_20240311_160540.xlsx
stock:         ./input/改价表更新 3-11.xlsx
combination:   ./input/替代料总表更新版3.5.xlsx
mrp:           ./input/US MRP20240311-运营.xlsx
group:         ./input/US-eBay 品类划分 2024.2.28.xlsx
last:          ./input/断货改价-us-上午 3.11.xlsx


# 2. 库存计算
## 2.1. 替代料库存

In [3]:
import pandas as pd

In [4]:
df_alternate = pd.read_excel(alternate,
                             header=0,
                             dtype=str).fillna('')

df_alternate['row'] = [i+1 for i in range(len(df_alternate))]

df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,替换料7,替换料8,替换料9,替换料10,row
0,USAC64980T,USAC64980,,,,,,,,,,1
1,USACC00055,USACC00055-C,,,,,,,,,,2
2,USACC01207-C,USACC01207,,,,,,,,,,3
3,USACC01311,USACC01311-C,,,,,,,,,,4
4,USACC01330S-C,USACC01330,USACC01330S-C1,,,,,,,,,5
...,...,...,...,...,...,...,...,...,...,...,...,...
109659,USGS40790U-C,,,,,,,,,,,109660
109660,USACC01710-C,,,,,,,,,,,109661
109661,USEOF29972-2-C,,,,,,,,,,,109662
109662,USEOF29972-4-C,,,,,,,,,,,109663


In [5]:
list_df = []
for column in df_alternate.columns.tolist()[:-1]:
    df_temp =  df_alternate[['主料SKU', column, 'row']]
    df_temp.columns = ['主料', 'SKU', 'row']
    df_temp = df_temp[df_temp['SKU'] != '']
    list_df.append(df_temp)

df_alternate = pd.concat(list_df, ignore_index=True)

df_alternate

,主料,SKU,row
0,USAC64980T,USAC64980T,1
1,USACC00055,USACC00055,2
2,USACC01207-C,USACC01207-C,3
3,USACC01311,USACC01311,4
4,USACC01330S-C,USACC01330S-C,5
...,...,...,...
247337,THB20022H,THB11102H,40890
247338,IC29037I-C,IC29037Z-C,67653
247339,ALT11462G-C,ALT11462A-FC,78530
247340,CA21685-C1,CA21685-C,88013


In [6]:
df_alternate[['SKU', 'row']].to_excel('./output/position.xlsx', index=False)

In [7]:
df_stock = pd.read_excel(stock,
                         sheet_name=0,
                         header=0,
                         names=['SKU', '库存'],
                         usecols='A, F',
                         dtype=str).fillna('')

df_stock['SKU'] = ['' if sku.strip().endswith('-NC') else sku.strip()[:-2] if sku.strip().endswith('-N') else sku.strip() for sku in df_stock['SKU'].tolist()]
df_stock = df_stock[df_stock['SKU'] != ''].reset_index(drop=True)
df_stock['库存'] = [int(stock.strip()) for stock in df_stock['库存'].tolist()]
df_stock = df_stock.groupby([df_stock['SKU']], as_index=False).sum()

df_stock

,SKU,库存
0,BCB96454-XH,0
1,BPD14040-XH,0
2,BPD8550086500+BRS31318,1
3,BRR34279-XH,0
4,CABM0021A,0
...,...,...
105661,YHTWS58202-Y-6,0
105662,YHTWS5W1ZBA-Y-10,0
105663,YHTWS5W1ZBA-Y-8,0
105664,YHTWS802AA-Y-6,0


In [8]:
print(df_alternate.columns)
print(df_stock.columns)

df_temp = pd.merge(left=df_alternate,
                   right=df_stock,
                   how='left',
                   left_on='SKU',
                   right_on='SKU').fillna(0)


df_temp

Index(['主料', 'SKU', 'row'], dtype='object')
Index(['SKU', '库存'], dtype='object')


,主料,SKU,row,库存
0,USAC64980T,USAC64980T,1,11.0
1,USACC00055,USACC00055,2,0.0
2,USACC01207-C,USACC01207-C,3,0.0
3,USACC01311,USACC01311,4,3.0
4,USACC01330S-C,USACC01330S-C,5,0.0
...,...,...,...,...
247337,THB20022H,THB11102H,40890,0.0
247338,IC29037I-C,IC29037Z-C,67653,0.0
247339,ALT11462G-C,ALT11462A-FC,78530,0.0
247340,CA21685-C1,CA21685-C,88013,0.0


In [9]:
df_temp = df_temp[['row', '库存']]
df_temp = df_temp.groupby([df_temp['row']], as_index=False).sum()

df_temp

,row,库存
0,1,11.0
1,2,6.0
2,3,0.0
3,4,10.0
4,5,6.0
...,...,...
109659,109660,0.0
109660,109661,0.0
109661,109662,0.0
109662,109663,0.0


In [10]:
print(df_alternate.columns)
print(df_stock.columns)

df_alternate = pd.merge(left=df_alternate,
                        right=df_temp,
                        how='left',
                        left_on='row',
                        right_on='row').fillna(0)

df_alternate

Index(['主料', 'SKU', 'row'], dtype='object')
Index(['SKU', '库存'], dtype='object')


,主料,SKU,row,库存
0,USAC64980T,USAC64980T,1,11.0
1,USACC00055,USACC00055,2,6.0
2,USACC01207-C,USACC01207-C,3,0.0
3,USACC01311,USACC01311,4,10.0
4,USACC01330S-C,USACC01330S-C,5,6.0
...,...,...,...,...
247337,THB20022H,THB11102H,40890,0.0
247338,IC29037I-C,IC29037Z-C,67653,0.0
247339,ALT11462G-C,ALT11462A-FC,78530,0.0
247340,CA21685-C1,CA21685-C,88013,0.0


In [11]:
print(df_stock.columns)
print(df_alternate.columns)

df_temp = pd.merge(left=df_stock,
                   right=df_alternate,
                   how='left',
                   left_on='SKU',
                   right_on='SKU',
                   suffixes=('', '_alternate')).fillna('')

df_temp = df_temp[df_temp['row'] == '']
df_temp['主料'] = df_temp['SKU'].tolist()
df_temp = df_temp[['主料', 'SKU', 'row', '库存']]

df_temp

Index(['SKU', '库存'], dtype='object')
Index(['主料', 'SKU', 'row', '库存'], dtype='object')


,主料,SKU,row,库存
0,BCB96454-XH,BCB96454-XH,,0
1,BPD14040-XH,BPD14040-XH,,0
2,BPD8550086500+BRS31318,BPD8550086500+BRS31318,,1
3,BRR34279-XH,BRR34279-XH,,0
4,CABM0021A,CABM0021A,,0
...,...,...,...,...
105661,YHTWS58202-Y-6,YHTWS58202-Y-6,,0
105662,YHTWS5W1ZBA-Y-10,YHTWS5W1ZBA-Y-10,,0
105663,YHTWS5W1ZBA-Y-8,YHTWS5W1ZBA-Y-8,,0
105664,YHTWS802AA-Y-6,YHTWS802AA-Y-6,,0


In [12]:
df_stock = pd.concat([df_alternate, df_temp], ignore_index=True)
df_stock['new_row'] = [i+1 for i in range(len(df_stock))]
df_stock['row'] = [row if row != '' else new_row for row, new_row in zip(df_stock['row'].tolist(), df_stock['new_row'].tolist())]
df_stock = df_stock[['主料', 'SKU', 'row', '库存']]

df_stock

,主料,SKU,row,库存
0,USAC64980T,USAC64980T,1,11.0
1,USACC00055,USACC00055,2,6.0
2,USACC01207-C,USACC01207-C,3,0.0
3,USACC01311,USACC01311,4,10.0
4,USACC01330S-C,USACC01330S-C,5,6.0
...,...,...,...,...
278264,YHTWS58202-Y-6,YHTWS58202-Y-6,278265,0.0
278265,YHTWS5W1ZBA-Y-10,YHTWS5W1ZBA-Y-10,278266,0.0
278266,YHTWS5W1ZBA-Y-8,YHTWS5W1ZBA-Y-8,278267,0.0
278267,YHTWS802AA-Y-6,YHTWS802AA-Y-6,278268,0.0


## 2.2. 组合库存

In [13]:
df_combination = pd.read_excel(combination,
                               header=1,
                               dtype=str)[['主料SKU', '数量', 'SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5']].fillna('')

df_combination['数量'] = [amount.strip() for amount in df_combination['数量'].tolist()]
df_combination = df_combination[(df_combination['数量'] != '') & (df_combination['数量'] != '1')].reset_index(drop=True)

df_combination

,主料SKU,数量,SKU1,SKU2,SKU3,SKU4,SKU5
0,USBDA04233-2-C,2,USBDA04233,,,,
1,USBDA26133-2,2,USBDA26133-C,,,,
2,USBDA26291-2-C,2,USBDA26291,,,,
3,USCAF10002-2-C,2,USCAF10002-C,,,,
4,USCAF10022-2-C,2,USCAF10022-C,,,,
...,...,...,...,...,...,...,...
511,USTCV18195LR-2-C,1+1,USTCV18195-C,USTCV17294,,,
512,USEAF12879-2-C,2,USEAF12879-C,,,,
513,USTCV16803IE-2-C,1+1,USTCV16803-C,USTCV18155-C,,,
514,USTCV18033IE-2-C,1+1,USTCV18033,USTCV18034,,,


In [14]:
print(df_combination.columns)
print(df_stock.columns)

df_combination = pd.merge(left=df_combination,
                          right=df_stock,
                          how='left',
                          left_on='主料SKU',
                          right_on='SKU').fillna('')

df_combination['new_row'] = [i+1 for i in range(len(df_combination))]
df_combination['new_new_row'] = [row if row != '' else new_row for row, new_row in zip(df_combination['row'].tolist(), df_combination['new_row'].tolist())]
df_combination = df_combination.drop_duplicates(['new_new_row'], keep='first', ignore_index=True)
df_combination = df_combination.drop_duplicates(['主料SKU'], keep='first', ignore_index=True)

df_combination

Index(['主料SKU', '数量', 'SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5'], dtype='object')
Index(['主料', 'SKU', 'row', '库存'], dtype='object')


,主料SKU,数量,SKU1,SKU2,SKU3,SKU4,SKU5,主料,SKU,row,库存,new_row,new_new_row
0,USBDA04233-2-C,2,USBDA04233,,,,,USBDA04233-2,USBDA04233-2-C,2703,62.0,1,2703
1,USBDA26133-2,2,USBDA26133-C,,,,,USBDA26133-2,USBDA26133-2,2888,17.0,2,2888
2,USBDA26291-2-C,2,USBDA26291,,,,,USBDA26291-2-C,USBDA26291-2-C,2889,12.0,3,2889
3,USCAF10002-2-C,2,USCAF10002-C,,,,,USCAF10002-2-C,USCAF10002-2-C,255028,4.0,4,255028
4,USCAF10022-2-C,2,USCAF10022-C,,,,,USCAF10022-2-C,USCAF10022-2-C,3436,7.0,5,3436
...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,USTCV18195LR-2-C,1+1,USTCV18195-C,USTCV17294,,,,USTCV18195LR-2-C,USTCV18195LR-2-C,272879,1.0,512,272879
511,USEAF12879-2-C,2,USEAF12879-C,,,,,USEAF12879-2-C1,USEAF12879-2-C,9282,14.0,513,9282
512,USTCV16803IE-2-C,1+1,USTCV16803-C,USTCV18155-C,,,,USTCV16803IE-2-C,USTCV16803IE-2-C,272747,0.0,514,272747
513,USTCV18033IE-2-C,1+1,USTCV18033,USTCV18034,,,,USTCV18033IE-2-C,USTCV18033IE-2-C,272831,0.0,515,272831


In [15]:
df_combination = pd.merge(left=df_combination,
                   right=df_stock[['SKU', '库存']],
                   how='left',
                   left_on='SKU1',
                   right_on='SKU',
                   suffixes=('', '_1')).fillna('')
df_combination = pd.merge(left=df_combination,
                   right=df_stock[['SKU', '库存']],
                   how='left',
                   left_on='SKU2',
                   right_on='SKU',
                   suffixes=('', '_2')).fillna('')
df_combination = pd.merge(left=df_combination,
                   right=df_stock[['SKU', '库存']],
                   how='left',
                   left_on='SKU3',
                   right_on='SKU',
                   suffixes=('', '_3')).fillna('')
df_combination = pd.merge(left=df_combination,
                   right=df_stock[['SKU', '库存']],
                   how='left',
                   left_on='SKU4',
                   right_on='SKU',
                   suffixes=('', '_4')).fillna('')
df_combination = pd.merge(left=df_combination,
                   right=df_stock[['SKU', '库存']],
                   how='left',
                   left_on='SKU5',
                   right_on='SKU',
                   suffixes=('', '_5')).fillna('')

df_combination

,主料SKU,数量,SKU1,SKU2,SKU3,SKU4,SKU5,主料,SKU,row,...,SKU_1,库存_1,SKU_2,库存_2,SKU_3,库存_3,SKU_4,库存_4,SKU_5,库存_5
0,USBDA04233-2-C,2,USBDA04233,,,,,USBDA04233-2,USBDA04233-2-C,2703,...,USBDA04233,11.0,,,,,,,,
1,USBDA26133-2,2,USBDA26133-C,,,,,USBDA26133-2,USBDA26133-2,2888,...,USBDA26133-C,27.0,,,,,,,,
2,USBDA26291-2-C,2,USBDA26291,,,,,USBDA26291-2-C,USBDA26291-2-C,2889,...,USBDA26291,5.0,,,,,,,,
3,USCAF10002-2-C,2,USCAF10002-C,,,,,USCAF10002-2-C,USCAF10002-2-C,255028,...,USCAF10002-C,27.0,,,,,,,,
4,USCAF10022-2-C,2,USCAF10022-C,,,,,USCAF10022-2-C,USCAF10022-2-C,3436,...,USCAF10022-C,26.0,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,USTCV18195LR-2-C,1+1,USTCV18195-C,USTCV17294,,,,USTCV18195LR-2-C,USTCV18195LR-2-C,272879,...,USTCV18195-C,1.0,USTCV17294,12.0,,,,,,
511,USEAF12879-2-C,2,USEAF12879-C,,,,,USEAF12879-2-C1,USEAF12879-2-C,9282,...,USEAF12879-C,16.0,,,,,,,,
512,USTCV16803IE-2-C,1+1,USTCV16803-C,USTCV18155-C,,,,USTCV16803IE-2-C,USTCV16803IE-2-C,272747,...,USTCV16803-C,0.0,USTCV18155-C,8.0,,,,,,
513,USTCV18033IE-2-C,1+1,USTCV18033,USTCV18034,,,,USTCV18033IE-2-C,USTCV18033IE-2-C,272831,...,USTCV18033,34.0,USTCV18034,18.0,,,,,,


In [16]:
from tqdm import tqdm
import math

In [17]:
for i in tqdm(range(len(df_combination)), desc='Progress', ncols=77):
    list_amount = [int(amount.strip()) for amount in df_combination.loc[i, '数量'].split('+')]
    list_stock = []
    for j in range(len(list_amount)):
        list_stock.append(0 if df_combination.loc[i, '库存_'+str(j+1)] == '' else df_combination.loc[i, '库存_'+str(j+1)] / list_amount[j])
    df_combination.loc[i, '组合库存'] = math.floor(min(list_stock))
        
df_combination = df_combination[['主料SKU', 'row', '组合库存']]

df_combination

Progress: 100%|██████████████████████████| 515/515 [00:00<00:00, 6221.39it/s]


,主料SKU,row,组合库存
0,USBDA04233-2-C,2703,5.0
1,USBDA26133-2,2888,13.0
2,USBDA26291-2-C,2889,2.0
3,USCAF10002-2-C,255028,13.0
4,USCAF10022-2-C,3436,13.0
...,...,...,...
510,USTCV18195LR-2-C,272879,1.0
511,USEAF12879-2-C,9282,8.0
512,USTCV16803IE-2-C,272747,0.0
513,USTCV18033IE-2-C,272831,18.0


In [18]:
print(df_stock.columns)
print(df_combination.columns)

df_temp_1 = pd.merge(left=df_stock,
                     right=df_combination[df_combination['row'] != ''],
                     how='left',
                     left_on='row',
                     right_on='row',
                     suffixes=('', '_combination')).fillna(0)

df_temp_1['库存'] = [stock+combination for stock, combination in zip(df_temp_1['库存'].tolist(), df_temp_1['组合库存'].tolist())]
df_temp_1 = df_temp_1[['主料', 'SKU', 'row', '库存']]

df_temp_1

Index(['主料', 'SKU', 'row', '库存'], dtype='object')
Index(['主料SKU', 'row', '组合库存'], dtype='object')


,主料,SKU,row,库存
0,USAC64980T,USAC64980T,1,11.0
1,USACC00055,USACC00055,2,6.0
2,USACC01207-C,USACC01207-C,3,0.0
3,USACC01311,USACC01311,4,10.0
4,USACC01330S-C,USACC01330S-C,5,6.0
...,...,...,...,...
278264,YHTWS58202-Y-6,YHTWS58202-Y-6,278265,0.0
278265,YHTWS5W1ZBA-Y-10,YHTWS5W1ZBA-Y-10,278266,0.0
278266,YHTWS5W1ZBA-Y-8,YHTWS5W1ZBA-Y-8,278267,0.0
278267,YHTWS802AA-Y-6,YHTWS802AA-Y-6,278268,0.0


In [19]:
df_temp_2 = df_combination[df_combination['row'] == '']
df_temp_2['主料'] = df_temp_2['主料SKU'].tolist()
df_temp_2['SKU'] = df_temp_2['主料SKU'].tolist()
df_temp_2['库存'] = df_temp_2['组合库存'].tolist()
df_temp_2 = df_temp_2[['主料', 'SKU', 'row', '库存']]

df_temp_2

,主料,SKU,row,库存


In [20]:
df_stock = pd.concat([df_temp_1, df_temp_2], ignore_index=True)
df_stock['new_row'] = [i+1 for i in range(len(df_stock))]
df_stock['row'] = [row if row != '' else new_row for row, new_row in zip(df_stock['row'].tolist(), df_stock['new_row'].tolist())]
df_stock = df_stock[['主料', 'SKU', 'row', '库存']]

df_stock

,主料,SKU,row,库存
0,USAC64980T,USAC64980T,1,11.0
1,USACC00055,USACC00055,2,6.0
2,USACC01207-C,USACC01207-C,3,0.0
3,USACC01311,USACC01311,4,10.0
4,USACC01330S-C,USACC01330S-C,5,6.0
...,...,...,...,...
278264,YHTWS58202-Y-6,YHTWS58202-Y-6,278265,0.0
278265,YHTWS5W1ZBA-Y-10,YHTWS5W1ZBA-Y-10,278266,0.0
278266,YHTWS5W1ZBA-Y-8,YHTWS5W1ZBA-Y-8,278267,0.0
278267,YHTWS802AA-Y-6,YHTWS802AA-Y-6,278268,0.0


# 3. MRP整理

In [24]:
df_mrp = pd.read_excel(mrp,
                       header=1,
                       names=['SKU', '品类', '层级', '身份', '未售出周数', '在途总数', 'W7', 'W6', 'W5', 'W4', 'W3', 'W2', 'W1', 'MRP周销'],
                       usecols='B, F, G, I, K, P, V:AB, AD',
                       dtype=str).fillna('')

df_group = pd.read_excel(group,
                         header=0,
                         dtype=str).fillna('')

df_mrp = pd.merge(left=df_mrp,
                  right=df_group[['品类', '品类负责人']],
                  how='left',
                  left_on='品类',
                  right_on='品类').fillna('')

df_mrp = df_mrp[df_mrp['品类负责人'] == 'Ada'].reset_index(drop=True)

df_mrp

,SKU,品类,层级,身份,未售出周数,在途总数,W7,W6,W5,W4,W3,W2,W1,MRP周销,品类负责人
0,USTCV17215HIE-2-C,TCV,TOP1,老品,0,102,26,35,39,36,40,41,63,46.3,Ada
1,USOBDAD100-C,OBD,TOP20,老品,0,229,0,0,9,12,21,44,55,40.6,Ada
2,USOBDTK100,OBD,TOP10,断货成长,0,0,79,30,0,1,18,47,36,31.74,Ada
3,USFP3688M0Z-C,FP,TOP1,老品,0,0,3,2,3,8,2,3,33,7.3,Ada
4,USET03102H-C,ET,TOP1,老品,0,84,5,11,11,10,13,29,31,23.8,Ada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13100,USTCV16000-2-C,TCV,普通,新品,0,6,0,0,0,0,0,0,0,0.5,Ada
13101,USTCV16000-C,TCV,普通,新品,0,12,0,0,0,0,0,0,0,1,Ada
13102,USTCV16000IE-4-C,TCV,TOP20,新品,0,6,0,0,0,0,0,0,0,0.5,Ada
13103,USTCV16000IE-2-C,TCV,普通,新品,0,6,0,0,0,0,0,0,0,0.5,Ada


In [25]:
print(df_mrp.columns)
print(df_stock.columns)

df_mrp = pd.merge(left=df_mrp,
                  right=df_stock[['SKU', 'row', '库存']],
                  how='left',
                  left_on='SKU',
                  right_on='SKU').fillna('')

df_mrp

Index(['SKU', '品类', '层级', '身份', '未售出周数', '在途总数', 'W7', 'W6', 'W5', 'W4', 'W3',
       'W2', 'W1', 'MRP周销', '品类负责人'],
      dtype='object')
Index(['主料', 'SKU', 'row', '库存'], dtype='object')


,SKU,品类,层级,身份,未售出周数,在途总数,W7,W6,W5,W4,W3,W2,W1,MRP周销,品类负责人,row,库存
0,USTCV17215HIE-2-C,TCV,TOP1,老品,0,102,26,35,39,36,40,41,63,46.3,Ada,8855,647.0
1,USOBDAD100-C,OBD,TOP20,老品,0,229,0,0,9,12,21,44,55,40.6,Ada,266782,219.0
2,USOBDTK100,OBD,TOP10,断货成长,0,0,79,30,0,1,18,47,36,31.74,Ada,266790,291.0
3,USFP3688M0Z-C,FP,TOP1,老品,0,0,3,2,3,8,2,3,33,7.3,Ada,5403,84.0
4,USET03102H-C,ET,TOP1,老品,0,84,5,11,11,10,13,29,31,23.8,Ada,4689,447.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13100,USTCV16000-2-C,TCV,普通,新品,0,6,0,0,0,0,0,0,0,0.5,Ada,272718,0.0
13101,USTCV16000-C,TCV,普通,新品,0,12,0,0,0,0,0,0,0,1,Ada,272719,0.0
13102,USTCV16000IE-4-C,TCV,TOP20,新品,0,6,0,0,0,0,0,0,0,0.5,Ada,272721,0.0
13103,USTCV16000IE-2-C,TCV,普通,新品,0,6,0,0,0,0,0,0,0,0.5,Ada,272720,0.0


In [26]:
df_temp = df_mrp.copy()
df_temp['times_row'] = [df_temp['row'].tolist().count(row) for row in df_temp['row'].tolist()]
df_temp['times_sku'] = [df_temp['SKU'].tolist().count(sku) for sku in df_temp['SKU'].tolist()]

df_temp

,SKU,品类,层级,身份,未售出周数,在途总数,W7,W6,W5,W4,W3,W2,W1,MRP周销,品类负责人,row,库存,times_row,times_sku
0,USTCV17215HIE-2-C,TCV,TOP1,老品,0,102,26,35,39,36,40,41,63,46.3,Ada,8855,647.0,1,1
1,USOBDAD100-C,OBD,TOP20,老品,0,229,0,0,9,12,21,44,55,40.6,Ada,266782,219.0,1,1
2,USOBDTK100,OBD,TOP10,断货成长,0,0,79,30,0,1,18,47,36,31.74,Ada,266790,291.0,1,1
3,USFP3688M0Z-C,FP,TOP1,老品,0,0,3,2,3,8,2,3,33,7.3,Ada,5403,84.0,1,1
4,USET03102H-C,ET,TOP1,老品,0,84,5,11,11,10,13,29,31,23.8,Ada,4689,447.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13100,USTCV16000-2-C,TCV,普通,新品,0,6,0,0,0,0,0,0,0,0.5,Ada,272718,0.0,1,1
13101,USTCV16000-C,TCV,普通,新品,0,12,0,0,0,0,0,0,0,1,Ada,272719,0.0,1,1
13102,USTCV16000IE-4-C,TCV,TOP20,新品,0,6,0,0,0,0,0,0,0,0.5,Ada,272721,0.0,1,1
13103,USTCV16000IE-2-C,TCV,普通,新品,0,6,0,0,0,0,0,0,0,0.5,Ada,272720,0.0,1,1


In [27]:
import datetime

In [28]:
df_temp_1 = df_temp[(df_temp['row'] != '') & (df_temp['times_row'] != 1)][['SKU', 'times_row']]
df_temp_1 = df_temp_1.drop_duplicates(ignore_index=True)
df_temp_1.columns = ['SKU', '出现次数']

df_temp_2 = df_temp[(df_temp['row'] == '') & (df_temp['times_sku'] != 1)][['SKU', 'times_sku']]
df_temp_2 = df_temp_2.drop_duplicates(ignore_index=True)
df_temp_2.columns = ['SKU', '出现次数']

df_temp_1 = pd.concat([df_temp_1, df_temp_2], ignore_index=True)

if len(df_temp_1) != 0:
    df_temp_1 = df_temp_1.sort_values(by=['SKU', '出现次数'], ascending=[True, True]).reset_index(drop=True)
    df_temp_1.to_excel('./output/error-mrp重复出现-'+ datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_1

,SKU,出现次数


In [29]:
import sys

In [30]:
if len(df_temp_1) != 0:
    sys.exit()

In [31]:
df_temp_2 = df_temp[df_temp['row'] == ''][['SKU']]

if len(df_temp_2) != 0:
    df_temp_2 = df_temp_2.sort_values(by=['SKU'], ascending=[True]).reset_index(drop=True)
    df_temp_2.to_excel('./output/warning-mrp缺失信息-'+ datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_2

,SKU


# 4. last与MRP整合

In [32]:
df_last = pd.read_excel(last,
                        header=1,
                        usecols='A, C:L, AE:AO, AQ:AU, AX:AY',
                        dtype=str).fillna('')

df_last['上次序号'] = [int(i.strip()) for i in df_last['序号'].tolist()]
df_last['最新断货时间'] = ['' if date == '00:00:00' or date == '' else date.split(' ')[0].replace('-', '/') for date in df_last['最新断货时间'].tolist()]

print(df_last.columns)

df_last

Index([                   '序号',                 '在线SKU',
                         'SKU',              '每周一大账号售价',
                        '第一涨价',                  '第二涨价',
                        '第三涨价',                  '第四涨价',
                        '第五涨价',                  '第六涨价',
                        '第七涨价',     2024-03-09 00:00:00,
           2024-03-08 00:00:00,     2024-03-07 00:00:01,
           2024-03-06 00:00:01,     2024-03-04 00:00:02,
           2024-03-05 00:00:02,     2024-03-06 00:00:02,
           2024-03-07 00:00:02, '2024-03-08 00:00:00.1',
       '2024-03-09 00:00:00.1',     2024-03-10 00:00:00,
                        '周销干预',                  '改价记录',
                      '本周改价记录',                '最新断货时间',
                       '品类负责人',                  '干预记录',
                          '提价',                  '上次序号'],
      dtype='object')


,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,2024-03-09 00:00:00.1,2024-03-10 00:00:00,周销干预,改价记录,本周改价记录,最新断货时间,品类负责人,干预记录,提价,上次序号
0,1,USVCS11099U,USVCS11099U,37.99,,,,,,,...,0,0,,1.4-T1; 2.12-H; 3.31-T0.5; 4.15-T1; 4.19-T0.5;...,,2023/03/11,Ada,,,1
1,2,USVCS11237,USVCS11237,26.49,,,,,,,...,1,0,,20.29-改0; 10.27-T5%; 1.3-H; 1.10-T2; 1.13-改0; ...,,2022/01/13,Ada,,,2
2,3,USVCS11064,USVCS11064U,,,,,,,,...,0,0,,1.26-T2; 2.21-T2; 3.1-T2; 3.7-T1; 3.10-H; 3.14...,,2023/03/17,Ada,,,3
3,4,USVCS11070,USVCS11070,35.5,,,,,,,...,0,0,,3.7-T1; 3.14-T3; 3.17-T4; 4.15-T2; 5.12-H; 3.2...,,2022/08/10,Ada,,,4
4,5,USVCS11516,USVCS11516,27.49,,,,,,,...,0,0,,3.31-T0.5; 7.20-H; 7.21-T5; 8.26-H; 10.13-T2; ...,,2023/01/09,Ada,,,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13097,13098,,USET03566-C,,,,,,,,...,0,0,,,,,,,,13098
13098,13099,,USFP13956-C,,,,,,,,...,0,0,,,,,,,,13099
13099,13100,,USACC11099-C,,,,,,,,...,0,0,,,,,,,,13100
13100,13101,,USTCV16000-2-C,,,,,,,,...,0,0,,,,,,,,13101


In [33]:
print(df_last.columns)
print(df_stock.columns)

df_last = pd.merge(left=df_last,
                   right=df_stock[['SKU', 'row', '库存']],
                   how='left',
                   left_on='SKU',
                   right_on='SKU').fillna('')

df_last

Index([                   '序号',                 '在线SKU',
                         'SKU',              '每周一大账号售价',
                        '第一涨价',                  '第二涨价',
                        '第三涨价',                  '第四涨价',
                        '第五涨价',                  '第六涨价',
                        '第七涨价',     2024-03-09 00:00:00,
           2024-03-08 00:00:00,     2024-03-07 00:00:01,
           2024-03-06 00:00:01,     2024-03-04 00:00:02,
           2024-03-05 00:00:02,     2024-03-06 00:00:02,
           2024-03-07 00:00:02, '2024-03-08 00:00:00.1',
       '2024-03-09 00:00:00.1',     2024-03-10 00:00:00,
                        '周销干预',                  '改价记录',
                      '本周改价记录',                '最新断货时间',
                       '品类负责人',                  '干预记录',
                          '提价',                  '上次序号'],
      dtype='object')
Index(['主料', 'SKU', 'row', '库存'], dtype='object')


,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,周销干预,改价记录,本周改价记录,最新断货时间,品类负责人,干预记录,提价,上次序号,row,库存
0,1,USVCS11099U,USVCS11099U,37.99,,,,,,,...,,1.4-T1; 2.12-H; 3.31-T0.5; 4.15-T1; 4.19-T0.5;...,,2023/03/11,Ada,,,1,8916,0.0
1,2,USVCS11237,USVCS11237,26.49,,,,,,,...,,20.29-改0; 10.27-T5%; 1.3-H; 1.10-T2; 1.13-改0; ...,,2022/01/13,Ada,,,2,273824,19.0
2,3,USVCS11064,USVCS11064U,,,,,,,,...,,1.26-T2; 2.21-T2; 3.1-T2; 3.7-T1; 3.10-H; 3.14...,,2023/03/17,Ada,,,3,8915,0.0
3,4,USVCS11070,USVCS11070,35.5,,,,,,,...,,3.7-T1; 3.14-T3; 3.17-T4; 4.15-T2; 5.12-H; 3.2...,,2022/08/10,Ada,,,4,273823,0.0
4,5,USVCS11516,USVCS11516,27.49,,,,,,,...,,3.31-T0.5; 7.20-H; 7.21-T5; 8.26-H; 10.13-T2; ...,,2023/01/09,Ada,,,5,273826,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13097,13098,,USET03566-C,,,,,,,,...,,,,,,,,13098,260307,0.0
13098,13099,,USFP13956-C,,,,,,,,...,,,,,,,,13099,261748,0.0
13099,13100,,USACC11099-C,,,,,,,,...,,,,,,,,13100,247433,0.0
13100,13101,,USTCV16000-2-C,,,,,,,,...,,,,,,,,13101,272718,0.0


In [34]:
df_temp = df_last.copy()
df_temp['times_row'] = [df_temp['row'].tolist().count(row) for row in df_temp['row'].tolist()]
df_temp['times_sku'] = [df_temp['SKU'].tolist().count(sku) for sku in df_temp['SKU'].tolist()]

df_temp

,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,本周改价记录,最新断货时间,品类负责人,干预记录,提价,上次序号,row,库存,times_row,times_sku
0,1,USVCS11099U,USVCS11099U,37.99,,,,,,,...,,2023/03/11,Ada,,,1,8916,0.0,1,1
1,2,USVCS11237,USVCS11237,26.49,,,,,,,...,,2022/01/13,Ada,,,2,273824,19.0,1,1
2,3,USVCS11064,USVCS11064U,,,,,,,,...,,2023/03/17,Ada,,,3,8915,0.0,1,1
3,4,USVCS11070,USVCS11070,35.5,,,,,,,...,,2022/08/10,Ada,,,4,273823,0.0,1,1
4,5,USVCS11516,USVCS11516,27.49,,,,,,,...,,2023/01/09,Ada,,,5,273826,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13097,13098,,USET03566-C,,,,,,,,...,,,,,,13098,260307,0.0,1,1
13098,13099,,USFP13956-C,,,,,,,,...,,,,,,13099,261748,0.0,1,1
13099,13100,,USACC11099-C,,,,,,,,...,,,,,,13100,247433,0.0,1,1
13100,13101,,USTCV16000-2-C,,,,,,,,...,,,,,,13101,272718,0.0,1,1


In [35]:
df_temp_1 = df_temp[(df_temp['row'] != '') & (df_temp['times_row'] != 1)][['SKU', 'times_row']]
df_temp_1 = df_temp_1.drop_duplicates(ignore_index=True)
df_temp_1.columns = ['SKU', '出现次数']

df_temp_2 = df_temp[(df_temp['row'] == '') & (df_temp['times_sku'] != 1)][['SKU', 'times_sku']]
df_temp_2 = df_temp_2.drop_duplicates(ignore_index=True)
df_temp_2.columns = ['SKU', '出现次数']

df_temp_1 = pd.concat([df_temp_1, df_temp_2], ignore_index=True)

if len(df_temp_1) != 0:
    df_temp_1 = df_temp_1.sort_values(by=['SKU', '出现次数'], ascending=[True, True]).reset_index(drop=True)
    df_temp_1.to_excel('./output/error-last重复出现-'+ datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_1

,SKU,出现次数


In [36]:
if len(df_temp_1) != 0:
    sys.exit()

In [37]:
df_temp_2 = df_temp[df_temp['row'] == ''][['SKU']]

if len(df_temp_2) != 0:
    df_temp_2 = df_temp_2.sort_values(by=['SKU'], ascending=[True]).reset_index(drop=True)
    df_temp_2.to_excel('./output/warning-last缺失信息-'+ datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_2

,SKU


In [38]:
print(df_last.columns)
print(df_mrp.columns)

df_temp_1 = pd.merge(left=df_last[df_last['row'] != ''],
                     right=df_mrp,
                     how='left',
                     left_on='row',
                     right_on='row',
                     suffixes=('', '_mrp')).fillna('')

df_temp_1['SKU'] = [sku if sku_mrp == '' else sku_mrp for sku, sku_mrp in zip(df_temp_1['SKU'].tolist(), df_temp_1['SKU_mrp'].tolist())]
df_temp_1['Note'] = ['' if charge == 'Ada' else 'last only, not in MRP' for charge in df_temp_1['品类负责人_mrp'].tolist()]
list_column = df_temp_1.columns.tolist()
list_column.remove('SKU_mrp')
list_column.remove('品类负责人_mrp')
list_column.remove('库存_mrp')
df_temp_1 = df_temp_1[list_column]

df_temp_1

Index([                   '序号',                 '在线SKU',
                         'SKU',              '每周一大账号售价',
                        '第一涨价',                  '第二涨价',
                        '第三涨价',                  '第四涨价',
                        '第五涨价',                  '第六涨价',
                        '第七涨价',     2024-03-09 00:00:00,
           2024-03-08 00:00:00,     2024-03-07 00:00:01,
           2024-03-06 00:00:01,     2024-03-04 00:00:02,
           2024-03-05 00:00:02,     2024-03-06 00:00:02,
           2024-03-07 00:00:02, '2024-03-08 00:00:00.1',
       '2024-03-09 00:00:00.1',     2024-03-10 00:00:00,
                        '周销干预',                  '改价记录',
                      '本周改价记录',                '最新断货时间',
                       '品类负责人',                  '干预记录',
                          '提价',                  '上次序号',
                         'row',                    '库存'],
      dtype='object')
Index(['SKU', '品类', '层级', '身份', '未售出周数', '在途总数', 'W7', 'W6', 'W5'

,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,在途总数,W7,W6,W5,W4,W3,W2,W1,MRP周销,Note
0,1,USVCS11099U,USVCS11099U,37.99,,,,,,,...,0,0,0,0,0,0,0,0,0,
1,2,USVCS11237,USVCS11237,26.49,,,,,,,...,11,0,0,3,2,1,1,2,1.17,
2,3,USVCS11064,USVCS11064U,,,,,,,,...,0,0,0,0,0,0,0,0,0,
3,4,USVCS11070,USVCS11070,35.5,,,,,,,...,0,0,0,0,0,0,0,0,0,
4,5,USVCS11516,USVCS11516,27.49,,,,,,,...,0,0,0,0,0,0,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13097,13098,,USET03566-C,,,,,,,,...,6,0,0,0,0,0,0,0,0.5,
13098,13099,,USFP13956-C,,,,,,,,...,12,0,0,0,0,0,0,0,1,
13099,13100,,USACC11099-C,,,,,,,,...,6,0,0,0,0,0,0,0,0.5,
13100,13101,,USTCV16000-2-C,,,,,,,,...,6,0,0,0,0,0,0,0,0.5,


In [39]:
print(df_last.columns)
print(df_mrp.columns)

df_temp_2 = pd.merge(left=df_last[df_last['row'] == ''],
                     right=df_mrp,
                     how='left',
                     left_on='SKU',
                     right_on='SKU',
                     suffixes=('', '_mrp')).fillna('')

df_temp_2['库存'] = [stock if stock_mrp == '' else stock_mrp for stock, stock_mrp in zip(df_temp_2['库存'].tolist(), df_temp_2['库存_mrp'].tolist())]
df_temp_2['Note'] = ['' if charge == 'Ada' else 'last only, not in MRP' for charge in df_temp_2['品类负责人_mrp'].tolist()]
list_column = df_temp_2.columns.tolist()
list_column.remove('品类负责人_mrp')
list_column.remove('row_mrp')
list_column.remove('库存_mrp')
df_temp_2 = df_temp_2[list_column]

df_temp_2

Index([                   '序号',                 '在线SKU',
                         'SKU',              '每周一大账号售价',
                        '第一涨价',                  '第二涨价',
                        '第三涨价',                  '第四涨价',
                        '第五涨价',                  '第六涨价',
                        '第七涨价',     2024-03-09 00:00:00,
           2024-03-08 00:00:00,     2024-03-07 00:00:01,
           2024-03-06 00:00:01,     2024-03-04 00:00:02,
           2024-03-05 00:00:02,     2024-03-06 00:00:02,
           2024-03-07 00:00:02, '2024-03-08 00:00:00.1',
       '2024-03-09 00:00:00.1',     2024-03-10 00:00:00,
                        '周销干预',                  '改价记录',
                      '本周改价记录',                '最新断货时间',
                       '品类负责人',                  '干预记录',
                          '提价',                  '上次序号',
                         'row',                    '库存'],
      dtype='object')
Index(['SKU', '品类', '层级', '身份', '未售出周数', '在途总数', 'W7', 'W6', 'W5'

,序号,在线SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,第七涨价,...,在途总数,W7,W6,W5,W4,W3,W2,W1,MRP周销,Note


In [40]:
df_temp_1 = pd.concat([df_temp_1, df_temp_2], ignore_index=True)

df_temp_1

,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,在途总数,W7,W6,W5,W4,W3,W2,W1,MRP周销,Note
0,1,USVCS11099U,USVCS11099U,37.99,,,,,,,...,0,0,0,0,0,0,0,0,0,
1,2,USVCS11237,USVCS11237,26.49,,,,,,,...,11,0,0,3,2,1,1,2,1.17,
2,3,USVCS11064,USVCS11064U,,,,,,,,...,0,0,0,0,0,0,0,0,0,
3,4,USVCS11070,USVCS11070,35.5,,,,,,,...,0,0,0,0,0,0,0,0,0,
4,5,USVCS11516,USVCS11516,27.49,,,,,,,...,0,0,0,0,0,0,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13097,13098,,USET03566-C,,,,,,,,...,6,0,0,0,0,0,0,0,0.5,
13098,13099,,USFP13956-C,,,,,,,,...,12,0,0,0,0,0,0,0,1,
13099,13100,,USACC11099-C,,,,,,,,...,6,0,0,0,0,0,0,0,0.5,
13100,13101,,USTCV16000-2-C,,,,,,,,...,6,0,0,0,0,0,0,0,0.5,


In [41]:
print(df_mrp.columns)
print(df_last.columns)

df_temp_2 = pd.merge(left=df_mrp,
                     right=df_last,
                     how='left',
                     left_on='SKU',
                     right_on='SKU',
                     suffixes=('', '_last')).fillna('')

df_temp_2 = df_temp_2[df_temp_2['序号'] == '']
df_temp_2['Note'] = 'MRP only, not in last'
list_column = df_temp_2.columns.tolist()
list_column.remove('品类负责人_last')
list_column.remove('row_last')
list_column.remove('库存_last')
df_temp_2 = df_temp_2[list_column]

df_temp_2

Index(['SKU', '品类', '层级', '身份', '未售出周数', '在途总数', 'W7', 'W6', 'W5', 'W4', 'W3',
       'W2', 'W1', 'MRP周销', '品类负责人', 'row', '库存'],
      dtype='object')
Index([                   '序号',                 '在线SKU',
                         'SKU',              '每周一大账号售价',
                        '第一涨价',                  '第二涨价',
                        '第三涨价',                  '第四涨价',
                        '第五涨价',                  '第六涨价',
                        '第七涨价',     2024-03-09 00:00:00,
           2024-03-08 00:00:00,     2024-03-07 00:00:01,
           2024-03-06 00:00:01,     2024-03-04 00:00:02,
           2024-03-05 00:00:02,     2024-03-06 00:00:02,
           2024-03-07 00:00:02, '2024-03-08 00:00:00.1',
       '2024-03-09 00:00:00.1',     2024-03-10 00:00:00,
                        '周销干预',                  '改价记录',
                      '本周改价记录',                '最新断货时间',
                       '品类负责人',                  '干预记录',
                          '提价',                  '

,SKU,品类,层级,身份,未售出周数,在途总数,W7,W6,W5,W4,...,2024-03-09 00:00:00.1,2024-03-10 00:00:00,周销干预,改价记录,本周改价记录,最新断货时间,干预记录,提价,上次序号,Note
131,USBDA04113KH-C,BDA,TOP10,老品,0,158,6,18,6,9,...,,,,,,,,,,"MRP only, not in last"
478,USET03576-C1,ET,TOP20,断货,0,11,0,0,0,0,...,,,,,,,,,,"MRP only, not in last"
931,USBMR3204K-C,BMR,TOP20,老品,0,31,0,2,0,2,...,,,,,,,,,,"MRP only, not in last"
1871,USBMR12992-C,BMR,普通,老品,0,29,0,1,0,0,...,,,,,,,,,,"MRP only, not in last"
1873,USBMR32438-C,BMR,普通,老品,0,2,0,1,1,1,...,,,,,,,,,,"MRP only, not in last"
4574,USBMR00089-C,BMR,普通,清仓,0,0,0,0,0,0,...,,,,,,,,,,"MRP only, not in last"
5628,USGS40790U-C,GS,普通,断货,0,9,1,0,0,1,...,,,,,,,,,,"MRP only, not in last"
12740,USACC01710-C,ACC,TOP20,新品,0,6,0,0,0,0,...,,,,,,,,,,"MRP only, not in last"
12917,USEOF29972-2-C,EOF,普通,新品,0,24,0,0,0,0,...,,,,,,,,,,"MRP only, not in last"
12918,USEOF29972-4-C,EOF,普通,新品,0,30,0,0,0,0,...,,,,,,,,,,"MRP only, not in last"


In [42]:
df = pd.concat([df_temp_1, df_temp_2], ignore_index=True)

df

,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,在途总数,W7,W6,W5,W4,W3,W2,W1,MRP周销,Note
0,1,USVCS11099U,USVCS11099U,37.99,,,,,,,...,0,0,0,0,0,0,0,0,0,
1,2,USVCS11237,USVCS11237,26.49,,,,,,,...,11,0,0,3,2,1,1,2,1.17,
2,3,USVCS11064,USVCS11064U,,,,,,,,...,0,0,0,0,0,0,0,0,0,
3,4,USVCS11070,USVCS11070,35.5,,,,,,,...,0,0,0,0,0,0,0,0,0,
4,5,USVCS11516,USVCS11516,27.49,,,,,,,...,0,0,0,0,0,0,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13115,,,USATF10200-C,,,,,,,,...,6,0,0,0,0,0,0,0,0.5,"MRP only, not in last"
13116,,,USATF58139-C,,,,,,,,...,12,0,0,0,0,0,0,0,1,"MRP only, not in last"
13117,,,USATF46100-C,,,,,,,,...,12,0,0,0,0,0,0,0,1,"MRP only, not in last"
13118,,,USATF28130-C,,,,,,,,...,6,0,0,0,0,0,0,0,0.5,"MRP only, not in last"


# 5. 其他计算

In [43]:
df_missing = pd.read_excel(stock,
                           sheet_name=1,
                           header=0,
                           names=['SKU', '数量'],
                           usecols='D, F',
                           dtype=str).fillna('')

df_missing['数量'] = [int(i.strip()) for i in df_missing['数量'].tolist()]
df_missing = df_missing.groupby([df_missing['SKU']], as_index=False).sum()

df_missing

,SKU,数量
0,USABS26220-C,1
1,USABS27080-C,2
2,USABS27170-C,2
3,USABS28300-C,2
4,USABS28860-C,1
...,...,...
151,USWR48942H-C,1
152,USWR49758-C,1
153,USWR49759-C,1
154,USWR51310-C,1


In [44]:
print(df_missing.columns)
print(df_stock.columns)

df_missing = pd.merge(left=df_missing,
                      right=df_stock,
                      how='left',
                      left_on='SKU',
                      right_on='SKU').fillna('')

df_missing

Index(['SKU', '数量'], dtype='object')
Index(['主料', 'SKU', 'row', '库存'], dtype='object')


,SKU,数量,主料,row,库存
0,USABS26220-C,1,USABS26220,72856,31.0
1,USABS27080-C,2,USABS27080-C,72382,21.0
2,USABS27170-C,2,USABS27170,72391,38.0
3,USABS28300-C,2,USABS28300-C,74695,0.0
4,USABS28860-C,1,USABS28860-C,72484,0.0
...,...,...,...,...,...
151,USWR48942H-C,1,USWR48942,59997,14.0
152,USWR49758-C,1,USWR49758-C,87270,14.0
153,USWR49759-C,1,USWR49759,87275,10.0
154,USWR51310-C,1,USWR51310,60069,44.0


In [45]:
df_temp = df_missing.copy()
df_temp['times'] = [df_temp['row'].tolist().count(row) for row in df_temp['row'].tolist()]

df_temp

,SKU,数量,主料,row,库存,times
0,USABS26220-C,1,USABS26220,72856,31.0,1
1,USABS27080-C,2,USABS27080-C,72382,21.0,1
2,USABS27170-C,2,USABS27170,72391,38.0,1
3,USABS28300-C,2,USABS28300-C,74695,0.0,1
4,USABS28860-C,1,USABS28860-C,72484,0.0,1
...,...,...,...,...,...,...
151,USWR48942H-C,1,USWR48942,59997,14.0,1
152,USWR49758-C,1,USWR49758-C,87270,14.0,1
153,USWR49759-C,1,USWR49759,87275,10.0,1
154,USWR51310-C,1,USWR51310,60069,44.0,1


In [46]:
df_temp_1 = df_temp[(df_temp['row'] != '') & (df_temp['times'] != 1)][['row', '数量']]
df_temp_1 = df_temp_1.groupby([df_temp_1['row']], as_index=False).sum()

df_temp_2 = df_temp[(df_temp['row'] != '') & (df_temp['times']) == 1][['row', '数量']]

df_temp_1 = pd.concat([df_temp_1, df_temp_2], ignore_index=True)

df_temp_1

,row,数量
0,72856,1
1,72382,2
2,72391,2
3,74695,2
4,72484,1
...,...,...
151,59997,1
152,87270,1
153,87275,1
154,60069,1


In [47]:
df_temp_2 = df_temp[df_temp['row'] == ''][['SKU', '数量']]
                                          
if len(df_temp_2) != 0:
    df_temp_2 = df_temp_2.sort_values(by=['SKU'], ascending=[True]).reset_index(drop=True)
    df_temp_2[['SKU']].to_excel('./output/warning-无record缺失信息-'+ datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_2

,SKU,数量


In [48]:
print(df.columns)
print(df_temp_1.columns)

df = pd.merge(left=df,
              right=df_temp_1,
              how='left',
              left_on='row',
              right_on='row').fillna(0)

df

Index([                   '序号',                 '在线SKU',
                         'SKU',              '每周一大账号售价',
                        '第一涨价',                  '第二涨价',
                        '第三涨价',                  '第四涨价',
                        '第五涨价',                  '第六涨价',
                        '第七涨价',     2024-03-09 00:00:00,
           2024-03-08 00:00:00,     2024-03-07 00:00:01,
           2024-03-06 00:00:01,     2024-03-04 00:00:02,
           2024-03-05 00:00:02,     2024-03-06 00:00:02,
           2024-03-07 00:00:02, '2024-03-08 00:00:00.1',
       '2024-03-09 00:00:00.1',     2024-03-10 00:00:00,
                        '周销干预',                  '改价记录',
                      '本周改价记录',                '最新断货时间',
                       '品类负责人',                  '干预记录',
                          '提价',                  '上次序号',
                         'row',                    '库存',
                          '品类',                    '层级',
                          '身份',

,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,W7,W6,W5,W4,W3,W2,W1,MRP周销,Note,数量
0,1,USVCS11099U,USVCS11099U,37.99,,,,,,,...,0,0,0,0,0,0,0,0,,0.0
1,2,USVCS11237,USVCS11237,26.49,,,,,,,...,0,0,3,2,1,1,2,1.17,,0.0
2,3,USVCS11064,USVCS11064U,,,,,,,,...,0,0,0,0,0,0,0,0,,0.0
3,4,USVCS11070,USVCS11070,35.5,,,,,,,...,0,0,0,0,0,0,0,0,,0.0
4,5,USVCS11516,USVCS11516,27.49,,,,,,,...,0,0,0,0,0,0,0,0,,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13115,,,USATF10200-C,,,,,,,,...,0,0,0,0,0,0,0,0.5,"MRP only, not in last",0.0
13116,,,USATF58139-C,,,,,,,,...,0,0,0,0,0,0,0,1,"MRP only, not in last",0.0
13117,,,USATF46100-C,,,,,,,,...,0,0,0,0,0,0,0,1,"MRP only, not in last",0.0
13118,,,USATF28130-C,,,,,,,,...,0,0,0,0,0,0,0,0.5,"MRP only, not in last",0.0


In [49]:
print(df.columns)
print(df_temp_2.columns)

df = pd.merge(left=df,
              right=df_temp_2,
              how='left',
              left_on='SKU',
              right_on='SKU',
              suffixes=('_1', '_2')).fillna(0)

df

Index([                   '序号',                 '在线SKU',
                         'SKU',              '每周一大账号售价',
                        '第一涨价',                  '第二涨价',
                        '第三涨价',                  '第四涨价',
                        '第五涨价',                  '第六涨价',
                        '第七涨价',     2024-03-09 00:00:00,
           2024-03-08 00:00:00,     2024-03-07 00:00:01,
           2024-03-06 00:00:01,     2024-03-04 00:00:02,
           2024-03-05 00:00:02,     2024-03-06 00:00:02,
           2024-03-07 00:00:02, '2024-03-08 00:00:00.1',
       '2024-03-09 00:00:00.1',     2024-03-10 00:00:00,
                        '周销干预',                  '改价记录',
                      '本周改价记录',                '最新断货时间',
                       '品类负责人',                  '干预记录',
                          '提价',                  '上次序号',
                         'row',                    '库存',
                          '品类',                    '层级',
                          '身份',

,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,W6,W5,W4,W3,W2,W1,MRP周销,Note,数量_1,数量_2
0,1,USVCS11099U,USVCS11099U,37.99,,,,,,,...,0,0,0,0,0,0,0,,0.0,0.0
1,2,USVCS11237,USVCS11237,26.49,,,,,,,...,0,3,2,1,1,2,1.17,,0.0,0.0
2,3,USVCS11064,USVCS11064U,,,,,,,,...,0,0,0,0,0,0,0,,0.0,0.0
3,4,USVCS11070,USVCS11070,35.5,,,,,,,...,0,0,0,0,0,0,0,,0.0,0.0
4,5,USVCS11516,USVCS11516,27.49,,,,,,,...,0,0,0,0,0,0,0,,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13115,,,USATF10200-C,,,,,,,,...,0,0,0,0,0,0,0.5,"MRP only, not in last",0.0,0.0
13116,,,USATF58139-C,,,,,,,,...,0,0,0,0,0,0,1,"MRP only, not in last",0.0,0.0
13117,,,USATF46100-C,,,,,,,,...,0,0,0,0,0,0,1,"MRP only, not in last",0.0,0.0
13118,,,USATF28130-C,,,,,,,,...,0,0,0,0,0,0,0.5,"MRP only, not in last",0.0,0.0


In [50]:
df['无Record ID'] = [amount_1+amount_2 for amount_1, amount_2 in zip(df['数量_1'].tolist(), df['数量_2'].tolist())]
df['当日库存'] = df['库存'].tolist()

list_column = df.columns.tolist()
list_column.remove('数量_1')
list_column.remove('数量_2')
list_column.remove('库存')
df = df[list_column]

df

,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,W6,W5,W4,W3,W2,W1,MRP周销,Note,无Record ID,当日库存
0,1,USVCS11099U,USVCS11099U,37.99,,,,,,,...,0,0,0,0,0,0,0,,0.0,0.0
1,2,USVCS11237,USVCS11237,26.49,,,,,,,...,0,3,2,1,1,2,1.17,,0.0,19.0
2,3,USVCS11064,USVCS11064U,,,,,,,,...,0,0,0,0,0,0,0,,0.0,0.0
3,4,USVCS11070,USVCS11070,35.5,,,,,,,...,0,0,0,0,0,0,0,,0.0,0.0
4,5,USVCS11516,USVCS11516,27.49,,,,,,,...,0,0,0,0,0,0,0,,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13115,,,USATF10200-C,,,,,,,,...,0,0,0,0,0,0,0.5,"MRP only, not in last",0.0,0.0
13116,,,USATF58139-C,,,,,,,,...,0,0,0,0,0,0,1,"MRP only, not in last",0.0,0.0
13117,,,USATF46100-C,,,,,,,,...,0,0,0,0,0,0,1,"MRP only, not in last",0.0,0.0
13118,,,USATF28130-C,,,,,,,,...,0,0,0,0,0,0,0.5,"MRP only, not in last",0.0,0.0


In [51]:
df['累计涨价'] = ['=SUM(F' + str(i+3) + ':L' + str(i+3) + ')' for i in range(len(df))]
df['近三个月平均销量'] = ['=IFERROR(ROUNDUP(AVERAGE(S' + str(i+3) + ':Y' + str(i+3) + '),1),"-")' for i in range(len(df))]
df['改价'] = ['=IFERROR(IF(AC' + str(i+3) + '-AE' + str(i+3) + '>0,"海运上架",IF(AC' + str(i+3) + '<=0,"断货","")),"")' for i in range(len(df))]
df['实时库存'] = ['=IFERROR(AD' + str(i+3) + '-AB' + str(i+3) + ',"")' for i in range(len(df))]
df['Week-Sale'] = ['=SUM(AI' + str(i+3) + ':AO' + str(i+3) + ')' for i in range(len(df))]
df['可周转数'] = ['=IFERROR(AC' + str(i+3) + '/AP' + str(i+3) + ',"")' for i in range(len(df))]
df['干预判断'] = ['=IFERROR(IFS(AV' + str(i+3) + '<4,"需要干预",AND(AV' + str(i+3) + '>4,AX' + str(i+3) + '<>""),"恢复干预"),"")' for i in range(len(df))]
df['改价记录1'] = ['=AR' + str(i+3) + '&AY' + str(i+3) for i in range(len(df))]
df['本周改价记录1'] = ['=AS' + str(i+3) + '&AY' + str(i+3) for i in range(len(df))]

df

C:\Users\YangTeng\AppData\Local\Temp\ipykernel_4224\3219910413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['累计涨价'] = ['=SUM(F' + str(i+3) + ':L' + str(i+3) + ')' for i in range(len(df))]
C:\Users\YangTeng\AppData\Local\Temp\ipykernel_4224\3219910413.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['近三个月平均销量'] = ['=IFERROR(ROUNDUP(AVERAGE(S' + str(i+3) + ':Y' + str(i+3) + '),1),"-")' for i in range(len(df))]


,序号,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,第六涨价,...,当日库存,累计涨价,近三个月平均销量,改价,实时库存,Week-Sale,可周转数,干预判断,改价记录1,本周改价记录1
0,1,USVCS11099U,USVCS11099U,37.99,,,,,,,...,0.0,=SUM(F3:L3),"=IFERROR(ROUNDUP(AVERAGE(S3:Y3),1),""-"")","=IFERROR(IF(AC3-AE3>0,""海运上架"",IF(AC3<=0,""断货"",""""...","=IFERROR(AD3-AB3,"""")",=SUM(AI3:AO3),"=IFERROR(AC3/AP3,"""")","=IFERROR(IFS(AV3<4,""需要干预"",AND(AV3>4,AX3<>""""),""...",=AR3&AY3,=AS3&AY3
1,2,USVCS11237,USVCS11237,26.49,,,,,,,...,19.0,=SUM(F4:L4),"=IFERROR(ROUNDUP(AVERAGE(S4:Y4),1),""-"")","=IFERROR(IF(AC4-AE4>0,""海运上架"",IF(AC4<=0,""断货"",""""...","=IFERROR(AD4-AB4,"""")",=SUM(AI4:AO4),"=IFERROR(AC4/AP4,"""")","=IFERROR(IFS(AV4<4,""需要干预"",AND(AV4>4,AX4<>""""),""...",=AR4&AY4,=AS4&AY4
2,3,USVCS11064,USVCS11064U,,,,,,,,...,0.0,=SUM(F5:L5),"=IFERROR(ROUNDUP(AVERAGE(S5:Y5),1),""-"")","=IFERROR(IF(AC5-AE5>0,""海运上架"",IF(AC5<=0,""断货"",""""...","=IFERROR(AD5-AB5,"""")",=SUM(AI5:AO5),"=IFERROR(AC5/AP5,"""")","=IFERROR(IFS(AV5<4,""需要干预"",AND(AV5>4,AX5<>""""),""...",=AR5&AY5,=AS5&AY5
3,4,USVCS11070,USVCS11070,35.5,,,,,,,...,0.0,=SUM(F6:L6),"=IFERROR(ROUNDUP(AVERAGE(S6:Y6),1),""-"")","=IFERROR(IF(AC6-AE6>0,""海运上架"",IF(AC6<=0,""断货"",""""...","=IFERROR(AD6-AB6,"""")",=SUM(AI6:AO6),"=IFERROR(AC6/AP6,"""")","=IFERROR(IFS(AV6<4,""需要干预"",AND(AV6>4,AX6<>""""),""...",=AR6&AY6,=AS6&AY6
4,5,USVCS11516,USVCS11516,27.49,,,,,,,...,0.0,=SUM(F7:L7),"=IFERROR(ROUNDUP(AVERAGE(S7:Y7),1),""-"")","=IFERROR(IF(AC7-AE7>0,""海运上架"",IF(AC7<=0,""断货"",""""...","=IFERROR(AD7-AB7,"""")",=SUM(AI7:AO7),"=IFERROR(AC7/AP7,"""")","=IFERROR(IFS(AV7<4,""需要干预"",AND(AV7>4,AX7<>""""),""...",=AR7&AY7,=AS7&AY7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13115,,,USATF10200-C,,,,,,,,...,0.0,=SUM(F13118:L13118),"=IFERROR(ROUNDUP(AVERAGE(S13118:Y13118),1),""-"")","=IFERROR(IF(AC13118-AE13118>0,""海运上架"",IF(AC1311...","=IFERROR(AD13118-AB13118,"""")",=SUM(AI13118:AO13118),"=IFERROR(AC13118/AP13118,"""")","=IFERROR(IFS(AV13118<4,""需要干预"",AND(AV13118>4,AX...",=AR13118&AY13118,=AS13118&AY13118
13116,,,USATF58139-C,,,,,,,,...,0.0,=SUM(F13119:L13119),"=IFERROR(ROUNDUP(AVERAGE(S13119:Y13119),1),""-"")","=IFERROR(IF(AC13119-AE13119>0,""海运上架"",IF(AC1311...","=IFERROR(AD13119-AB13119,"""")",=SUM(AI13119:AO13119),"=IFERROR(AC13119/AP13119,"""")","=IFERROR(IFS(AV13119<4,""需要干预"",AND(AV13119>4,AX...",=AR13119&AY13119,=AS13119&AY13119
13117,,,USATF46100-C,,,,,,,,...,0.0,=SUM(F13120:L13120),"=IFERROR(ROUNDUP(AVERAGE(S13120:Y13120),1),""-"")","=IFERROR(IF(AC13120-AE13120>0,""海运上架"",IF(AC1312...","=IFERROR(AD13120-AB13120,"""")",=SUM(AI13120:AO13120),"=IFERROR(AC13120/AP13120,"""")","=IFERROR(IFS(AV13120<4,""需要干预"",AND(AV13120>4,AX...",=AR13120&AY13120,=AS13120&AY13120
13118,,,USATF28130-C,,,,,,,,...,0.0,=SUM(F13121:L13121),"=IFERROR(ROUNDUP(AVERAGE(S13121:Y13121),1),""-"")","=IFERROR(IF(AC13121-AE13121>0,""海运上架"",IF(AC1312...","=IFERROR(AD13121-AB13121,"""")",=SUM(AI13121:AO13121),"=IFERROR(AC13121/AP13121,"""")","=IFERROR(IFS(AV13121<4,""需要干预"",AND(AV13121>4,AX...",=AR13121&AY13121,=AS13121&AY13121


In [52]:
list_column = df.columns.tolist()
df = df[['序号', '层级', '在线SKU', 'SKU', '每周一大账号售价', '第一涨价', '第二涨价', '第三涨价', '第四涨价', '第五涨价', '第六涨价', '第七涨价', '累计涨价',
         '品类', '身份', 'MRP周销', '近三个月平均销量', '未售出周数', 'W7', 'W6', 'W5', 'W4', 'W3', 'W2', 'W1',
         '在途总数', '改价', '无Record ID', '实时库存', '当日库存'] +
        list_column[11:22] +
        ['Week-Sale', '周销干预', '改价记录', '本周改价记录', '最新断货时间', '品类负责人', '可周转数', '干预判断', '干预记录',
         '提价', '改价记录1', '本周改价记录1', 'Note', '上次序号']]

df.sort_values(by=['上次序号', 'SKU'], ascending=[True, True]).reset_index(drop=True)
df['序号'] = [i+1 for i in range(len(df))]

df

C:\Users\YangTeng\AppData\Local\Temp\ipykernel_4224\819767609.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['序号'] = [i+1 for i in range(len(df))]


,序号,层级,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,...,最新断货时间,品类负责人,可周转数,干预判断,干预记录,提价,改价记录1,本周改价记录1,Note,上次序号
0,1,普通,USVCS11099U,USVCS11099U,37.99,,,,,,...,2023/03/11,Ada,"=IFERROR(AC3/AP3,"""")","=IFERROR(IFS(AV3<4,""需要干预"",AND(AV3>4,AX3<>""""),""...",,,=AR3&AY3,=AS3&AY3,,1
1,2,普通,USVCS11237,USVCS11237,26.49,,,,,,...,2022/01/13,Ada,"=IFERROR(AC4/AP4,"""")","=IFERROR(IFS(AV4<4,""需要干预"",AND(AV4>4,AX4<>""""),""...",,,=AR4&AY4,=AS4&AY4,,2
2,3,普通,USVCS11064,USVCS11064U,,,,,,,...,2023/03/17,Ada,"=IFERROR(AC5/AP5,"""")","=IFERROR(IFS(AV5<4,""需要干预"",AND(AV5>4,AX5<>""""),""...",,,=AR5&AY5,=AS5&AY5,,3
3,4,普通,USVCS11070,USVCS11070,35.5,,,,,,...,2022/08/10,Ada,"=IFERROR(AC6/AP6,"""")","=IFERROR(IFS(AV6<4,""需要干预"",AND(AV6>4,AX6<>""""),""...",,,=AR6&AY6,=AS6&AY6,,4
4,5,普通,USVCS11516,USVCS11516,27.49,,,,,,...,2023/01/09,Ada,"=IFERROR(AC7/AP7,"""")","=IFERROR(IFS(AV7<4,""需要干预"",AND(AV7>4,AX7<>""""),""...",,,=AR7&AY7,=AS7&AY7,,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13115,13116,普通,,USATF10200-C,,,,,,,...,,Ada,"=IFERROR(AC13118/AP13118,"""")","=IFERROR(IFS(AV13118<4,""需要干预"",AND(AV13118>4,AX...",,,=AR13118&AY13118,=AS13118&AY13118,"MRP only, not in last",
13116,13117,普通,,USATF58139-C,,,,,,,...,,Ada,"=IFERROR(AC13119/AP13119,"""")","=IFERROR(IFS(AV13119<4,""需要干预"",AND(AV13119>4,AX...",,,=AR13119&AY13119,=AS13119&AY13119,"MRP only, not in last",
13117,13118,普通,,USATF46100-C,,,,,,,...,,Ada,"=IFERROR(AC13120/AP13120,"""")","=IFERROR(IFS(AV13120<4,""需要干预"",AND(AV13120>4,AX...",,,=AR13120&AY13120,=AS13120&AY13120,"MRP only, not in last",
13118,13119,普通,,USATF28130-C,,,,,,,...,,Ada,"=IFERROR(AC13121/AP13121,"""")","=IFERROR(IFS(AV13121<4,""需要干预"",AND(AV13121>4,AX...",,,=AR13121&AY13121,=AS13121&AY13121,"MRP only, not in last",


In [53]:
for i in tqdm(range(len(df)), desc='Progress', ncols=77):
    try:
        df.loc[i, '每周一大账号售价'] = float(df.loc[i, '每周一大账号售价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第一涨价'] = float(df.loc[i, '第一涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第二涨价'] = float(df.loc[i, '第二涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第三涨价'] = float(df.loc[i, '第三涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第四涨价'] = float(df.loc[i, '第四涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第五涨价'] = float(df.loc[i, '第五涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第六涨价'] = float(df.loc[i, '第六涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第七涨价'] = float(df.loc[i, '第七涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'MRP周销'] = float(df.loc[i, 'MRP周销'].strip())
    except:
        pass
    
    try:
        df.loc[i, '未售出周数'] = int(df.loc[i, '未售出周数'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W7'] = int(df.loc[i, 'W7'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W6'] = int(df.loc[i, 'W6'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W5'] = int(df.loc[i, 'W5'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W4'] = int(df.loc[i, 'W4'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W3'] = int(df.loc[i, 'W3'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W2'] = int(df.loc[i, 'W2'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W1'] = int(df.loc[i, 'W1'].strip())
    except:
        pass
    
    try:
        df.loc[i, '在途总数'] = int(df.loc[i, '在途总数'].strip())
    except:
        pass
    
    try:
        df.loc[i, '无Record ID'] = int(df.loc[i, '无Record ID'].strip())
    except:
        pass
    
    try:
        df.loc[i, '当日库存'] = int(df.loc[i, '当日库存'].strip())
    except:
        pass
    
    try:
        df.iloc[i, 30] = int(df.iloc[i, 30].strip())
    except:
        pass
    
    try:
        df.iloc[i, 31] = int(df.iloc[i, 31].strip())
    except:
        pass
    
    try:
        df.iloc[i, 32] = int(df.iloc[i, 32].strip())
    except:
        pass
    
    try:
        df.iloc[i, 33] = int(df.iloc[i, 33].strip())
    except:
        pass
    
    try:
        df.iloc[i, 34] = int(df.iloc[i, 34].strip())
    except:
        pass
    
    try:
        df.iloc[i, 35] = int(df.iloc[i, 35].strip())
    except:
        pass
    
    try:
        df.iloc[i, 36] = int(df.iloc[i, 36].strip())
    except:
        pass
    
    try:
        df.iloc[i, 37] = int(df.iloc[i, 37].strip())
    except:
        pass
    
    try:
        df.iloc[i, 38] = int(df.iloc[i, 38].strip())
    except:
        pass
    
    try:
        df.iloc[i, 39] = int(df.iloc[i, 39].strip())
    except:
        pass
    
    try:
        df.iloc[i, 40] = int(df.iloc[i, 40].strip())
    except:
        pass
    
df.to_excel('./output/pm-colorless.xlsx', index=False, startrow=1)

df

Progress: 100%|███████████████████████| 13120/13120 [00:28<00:00, 468.40it/s]


,序号,层级,在线SKU,SKU,每周一大账号售价,第一涨价,第二涨价,第三涨价,第四涨价,第五涨价,...,最新断货时间,品类负责人,可周转数,干预判断,干预记录,提价,改价记录1,本周改价记录1,Note,上次序号
0,1,普通,USVCS11099U,USVCS11099U,37.99,,,,,,...,2023/03/11,Ada,"=IFERROR(AC3/AP3,"""")","=IFERROR(IFS(AV3<4,""需要干预"",AND(AV3>4,AX3<>""""),""...",,,=AR3&AY3,=AS3&AY3,,1
1,2,普通,USVCS11237,USVCS11237,26.49,,,,,,...,2022/01/13,Ada,"=IFERROR(AC4/AP4,"""")","=IFERROR(IFS(AV4<4,""需要干预"",AND(AV4>4,AX4<>""""),""...",,,=AR4&AY4,=AS4&AY4,,2
2,3,普通,USVCS11064,USVCS11064U,,,,,,,...,2023/03/17,Ada,"=IFERROR(AC5/AP5,"""")","=IFERROR(IFS(AV5<4,""需要干预"",AND(AV5>4,AX5<>""""),""...",,,=AR5&AY5,=AS5&AY5,,3
3,4,普通,USVCS11070,USVCS11070,35.5,,,,,,...,2022/08/10,Ada,"=IFERROR(AC6/AP6,"""")","=IFERROR(IFS(AV6<4,""需要干预"",AND(AV6>4,AX6<>""""),""...",,,=AR6&AY6,=AS6&AY6,,4
4,5,普通,USVCS11516,USVCS11516,27.49,,,,,,...,2023/01/09,Ada,"=IFERROR(AC7/AP7,"""")","=IFERROR(IFS(AV7<4,""需要干预"",AND(AV7>4,AX7<>""""),""...",,,=AR7&AY7,=AS7&AY7,,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13115,13116,普通,,USATF10200-C,,,,,,,...,,Ada,"=IFERROR(AC13118/AP13118,"""")","=IFERROR(IFS(AV13118<4,""需要干预"",AND(AV13118>4,AX...",,,=AR13118&AY13118,=AS13118&AY13118,"MRP only, not in last",
13116,13117,普通,,USATF58139-C,,,,,,,...,,Ada,"=IFERROR(AC13119/AP13119,"""")","=IFERROR(IFS(AV13119<4,""需要干预"",AND(AV13119>4,AX...",,,=AR13119&AY13119,=AS13119&AY13119,"MRP only, not in last",
13117,13118,普通,,USATF46100-C,,,,,,,...,,Ada,"=IFERROR(AC13120/AP13120,"""")","=IFERROR(IFS(AV13120<4,""需要干预"",AND(AV13120>4,AX...",,,=AR13120&AY13120,=AS13120&AY13120,"MRP only, not in last",
13118,13119,普通,,USATF28130-C,,,,,,,...,,Ada,"=IFERROR(AC13121/AP13121,"""")","=IFERROR(IFS(AV13121<4,""需要干预"",AND(AV13121>4,AX...",,,=AR13121&AY13121,=AS13121&AY13121,"MRP only, not in last",


# 6. 格式更改

In [54]:
from openpyxl import load_workbook

In [55]:
wb_last = load_workbook(last)
ws_last = wb_last.worksheets[0]

wb_this = load_workbook('./output/pm-colorless.xlsx')
ws_this = wb_this.worksheets[0]

In [56]:
coef = 1.15
ws_this.column_dimensions['A'].width = coef * 6
ws_this.column_dimensions['B'].width = coef * 9
ws_this.column_dimensions['C'].width = coef * 15
ws_this.column_dimensions['D'].width = coef * 15
ws_this.column_dimensions['E'].width = coef * 9
ws_this.column_dimensions['F'].width = coef * 8
ws_this.column_dimensions['G'].width = coef * 8
ws_this.column_dimensions['H'].width = coef * 8
ws_this.column_dimensions['I'].width = coef * 8
ws_this.column_dimensions['J'].width = coef * 8
ws_this.column_dimensions['K'].width = coef * 8
ws_this.column_dimensions['L'].width = coef * 8
ws_this.column_dimensions['M'].width = coef * 9
ws_this.column_dimensions['N'].width = coef * 9
ws_this.column_dimensions['O'].width = coef * 9
ws_this.column_dimensions['P'].width = coef * 7
ws_this.column_dimensions['Q'].width = coef * 7
ws_this.column_dimensions['R'].width = coef * 5
ws_this.column_dimensions['S'].width = coef * 5
ws_this.column_dimensions['T'].width = coef * 5
ws_this.column_dimensions['U'].width = coef * 5
ws_this.column_dimensions['V'].width = coef * 5
ws_this.column_dimensions['W'].width = coef * 5
ws_this.column_dimensions['X'].width = coef * 5
ws_this.column_dimensions['Y'].width = coef * 5
ws_this.column_dimensions['Z'].width = coef * 7
ws_this.column_dimensions['AA'].width = coef * 7
ws_this.column_dimensions['AB'].width = coef * 10
ws_this.column_dimensions['AC'].width = coef * 10
ws_this.column_dimensions['AD'].width = coef * 10
ws_this.column_dimensions['AE'].width = coef * 8
ws_this.column_dimensions['AF'].width = coef * 8
ws_this.column_dimensions['AG'].width = coef * 8
ws_this.column_dimensions['AH'].width = coef * 8
ws_this.column_dimensions['AI'].width = coef * 8
ws_this.column_dimensions['AJ'].width = coef * 8
ws_this.column_dimensions['AK'].width = coef * 8
ws_this.column_dimensions['AL'].width = coef * 8
ws_this.column_dimensions['AM'].width = coef * 8
ws_this.column_dimensions['AN'].width = coef * 8
ws_this.column_dimensions['AO'].width = coef * 8
ws_this.column_dimensions['AP'].width = coef * 8
ws_this.column_dimensions['AQ'].width = coef * 10
ws_this.column_dimensions['AR'].width = coef * 10
ws_this.column_dimensions['AS'].width = coef * 10
ws_this.column_dimensions['AT'].width = coef * 10
ws_this.column_dimensions['AU'].width = coef * 8
ws_this.column_dimensions['AV'].width = coef * 8
ws_this.column_dimensions['AW'].width = coef * 8
ws_this.column_dimensions['AX'].width = coef * 8
ws_this.column_dimensions['AY'].width = coef * 8
ws_this.column_dimensions['AZ'].width = coef * 8
ws_this.column_dimensions['BA'].width = coef * 8
ws_this.column_dimensions['BB'].width = coef * 10
ws_this.column_dimensions['BC'].width = coef * 6

ws_this.row_dimensions[1].height = 25
ws_this.row_dimensions[2].height = 34
for i in range(3, ws_this.max_row+1):
    ws_this.row_dimensions[i].height = 16.5

In [57]:
from openpyxl.styles import Font, PatternFill, Alignment

In [58]:
ws_this['A1'].value = str(datetime.date.today()).replace('-', '/')
ws_this['A1'].font = Font(b=True)
ws_this['A1'].number_format = 'm/d;@'
ws_this['A1'].alignment = Alignment(shrinkToFit=True)
ws_this.merge_cells('A1:B1')

ws_this['C1'].value = '每周二删除本周改价记录栏'
ws_this['C1'].font = Font(bold=True, color='FFFF0000')
ws_this['C1'].alignment = Alignment(shrinkToFit=True)
ws_this['C1'].fill = PatternFill(patternType='solid', fgColor='FFFFFF00')

ws_this['D1'].value = '基本信息'
ws_this['D1'].font = Font(b=True)
ws_this['D1'].alignment = Alignment(shrinkToFit=True)
ws_this.merge_cells('D1:R1')

ws_this['S1'].value = '近7周 周销情况'
ws_this['S1'].alignment = Alignment(shrinkToFit=True)
ws_this.merge_cells('S1:Y1')

ws_this['AA1'].value = '任务栏'
ws_this['AA1'].font = Font(b=True)
ws_this['AA1'].alignment = Alignment(shrinkToFit=True)
ws_this['AA1'].fill = PatternFill(patternType='solid', fgColor='FF92D050')

ws_this['AB1'].value = '=SUM(AB3:AB' + str(ws_this.max_row) + ')'
ws_this['AB1'].font = Font(b=True, color='FFFF0000')
ws_this['AB1'].alignment = Alignment(shrinkToFit=True)

ws_this['AC1'].value = '=SUM(AC3:AC' + str(ws_this.max_row) + ')'
ws_this['AC1'].font = Font(b=True, color='FFFF0000')
ws_this['AC1'].alignment = Alignment(shrinkToFit=True)

ws_this['AD1'].value = '=SUM(AD3:AD' + str(ws_this.max_row) + ')'
ws_this['AD1'].font = Font(b=True, color='FFFF0000')
ws_this['AD1'].alignment = Alignment(shrinkToFit=True)

ws_this['AE1'].value = '=SUM(AE3:AE' + str(ws_this.max_row) + ')'
ws_this['AE1'].font = Font(b=True, color='FFFF0000')
ws_this['AE1'].alignment = Alignment(shrinkToFit=True)

ws_this['AF1'].value = '=SUM(AF3:AF' + str(ws_this.max_row) + ')'
ws_this['AF1'].font = Font(b=True, color='FFFF0000')
ws_this['AF1'].alignment = Alignment(shrinkToFit=True)

ws_this['AG1'].value = '=SUM(AG3:AG' + str(ws_this.max_row) + ')'
ws_this['AG1'].font = Font(b=True, color='FFFF0000')
ws_this['AG1'].alignment = Alignment(shrinkToFit=True)

ws_this['AH1'].value = '=SUM(AH3:AH' + str(ws_this.max_row) + ')'
ws_this['AH1'].font = Font(b=True, color='FFFF0000')
ws_this['AH1'].alignment = Alignment(shrinkToFit=True)

ws_this['AR1'].value = '格式案例：10.19-改0; 9.23-T5; 9.30-H; 10.5-J3; 10.18-T6(日期，最新在后面）'
ws_this['AR1'].alignment = Alignment(shrinkToFit=True)

ws_this['AV1'].value = '实时库存≤5，周销不为0，或者周销为0，无record不为0的，在线都已改为实时库存，且已关闭自动补货'
ws_this['AV1'].alignment = Alignment(shrinkToFit=True)
ws_this['AV1'].fill = PatternFill(patternType='solid', fgColor='FFFFFF00')

In [59]:
for list_cell in ws_this['A2:R2']:
    for cell in list_cell:
        cell.alignment = Alignment(shrinkToFit=True)
        cell.fill = PatternFill(patternType='solid', fgColor='FF8DB4E2')

for list_cell in ws_this['S2:Y2']:
    for cell in list_cell:
        cell.alignment = Alignment(shrinkToFit=True)
        
for list_cell in ws_this['Z2:AA2']:
    for cell in list_cell:
        cell.alignment = Alignment(shrinkToFit=True)
        cell.fill = PatternFill(patternType='solid', fgColor='FF8DB4E2')
        
ws_this['AB2'].alignment = Alignment(shrinkToFit=True)
ws_this['AB2'].fill = PatternFill(patternType='solid', fgColor='FFFFFF00')

ws_this['AC2'].alignment = Alignment(shrinkToFit=True)
ws_this['AC2'].fill = PatternFill(patternType='solid', fgColor='FF8DB4E2')

ws_this['AD2'].alignment = Alignment(shrinkToFit=True)
ws_this['AD2'].fill = PatternFill(patternType='solid', fgColor='FF00B0F0')

for list_cell in ws_this['AE2:AH2']:
    for cell in list_cell:
        cell.number_format = 'm/d;@'
        cell.alignment = Alignment(shrinkToFit=True)
        cell.fill = PatternFill(patternType='solid', fgColor='FF00B0F0')

for list_cell in ws_this['AI2:AO2']:
    for cell in list_cell:
        cell.number_format = 'm/d;@'
        cell.alignment = Alignment(shrinkToFit=True)
        cell.fill = PatternFill(patternType='solid', fgColor='FFFABF8E')

for list_cell in ws_this['AP2:AQ2']:
    for cell in list_cell:
        cell.font = Font(color='FFFFFFFF')
        cell.alignment = Alignment(shrinkToFit=True)
        cell.fill = PatternFill(patternType='solid', fgColor='FF963634')

for list_cell in ws_this['AR2:AU2']:
    for cell in list_cell:
        cell.alignment = Alignment(shrinkToFit=True)
        cell.fill = PatternFill(patternType='solid', fgColor='FF8DB4E2')

for list_cell in ws_this['AV2:AX2']:
    for cell in list_cell:
        cell.font = Font(color='FFFFFFFF')
        cell.alignment = Alignment(shrinkToFit=True)
        cell.fill = PatternFill(patternType='solid', fgColor='FFE26B0A')

for list_cell in ws_this['AY2:BA2']:
    for cell in list_cell:
        cell.alignment = Alignment(shrinkToFit=True)
        cell.fill = PatternFill(patternType='solid', fgColor='FF8DB4E2')
        
for list_cell in ws_this['BB2:BC2']:
    for cell in list_cell:
        cell.alignment = Alignment(shrinkToFit=True)

In [60]:
for i in tqdm(range(3, ws_this.max_row+1), desc='Progress', ncols=77):
    ws_this['E' + str(i)].number_format = '\\$#,##0.00;\\-\\$#,##0.00'
    
    ws_this['AQ' + str(i)].alignment = Alignment(shrinkToFit=True)
    # ws_this['AR' + str(i)].alignment = Alignment(shrinkToFit=True)
    ws_this['AS' + str(i)].alignment = Alignment(shrinkToFit=True)
    ws_this['AV' + str(i)].alignment = Alignment(shrinkToFit=True)
    ws_this['AW' + str(i)].alignment = Alignment(shrinkToFit=True)
    ws_this['AX' + str(i)].alignment = Alignment(shrinkToFit=True)
    ws_this['AY' + str(i)].alignment = Alignment(shrinkToFit=True)
    ws_this['AZ' + str(i)].alignment = Alignment(shrinkToFit=True)
    ws_this['BA' + str(i)].alignment = Alignment(shrinkToFit=True)
    ws_this['BB' + str(i)].alignment = Alignment(shrinkToFit=True)
    
    no = ws_this['BC' + str(i)].value
    if no == None:
        continue
    
    no = int(no) + 2
    if ws_last['AH' + str(no)].font.color != None and type(ws_last['AH' + str(no)].font.color.rgb) == str:
        ws_this['AH' + str(i)].font = Font(color=ws_last['AH' + str(no)].font.color.rgb)
    if ws_last['AG' + str(no)].font.color != None and type(ws_last['AG' + str(no)].font.color.rgb) == str:
        ws_this['AG' + str(i)].font = Font(color=ws_last['AG' + str(no)].font.color.rgb)
    if ws_last['AF' + str(no)].font.color != None and type(ws_last['AF' + str(no)].font.color.rgb) == str:
        ws_this['AF' + str(i)].font = Font(color=ws_last['AF' + str(no)].font.color.rgb)
    if ws_last['AE' + str(no)].font.color != None and type(ws_last['AE' + str(no)].font.color.rgb) == str:
        ws_this['AE' + str(i)].font = Font(color=ws_last['AE' + str(no)].font.color.rgb)
    if ws_last['AD' + str(no)].font.color != None and type(ws_last['AD' + str(no)].font.color.rgb) == str:
        ws_this['AD' + str(i)].font = Font(color=ws_last['AD' + str(no)].font.color.rgb)

Progress: 100%|██████████████████████| 13120/13120 [00:04<00:00, 2737.22it/s]


In [61]:
wb_this.save('./断货改价-us-下午.xlsx')
wb_last.close()
wb_this.close()

print('Done ~')

Done ~
